>#                     Web Scraping de E-commerce

A fim de praticar os bons hábitos de programação, respeitando todos os ordenamentos, realizo uma breve 'raspagem' de Dados em uma <a href="http://books.toscrape.com/">Loja</a> online de Livros.

In [1]:
#Importando algumas bibliotecas
import pandas as pd 
import numpy as np
import requests
import re
from bs4 import BeautifulSoup

In [2]:
def web_search(url_search):
    """Função para extrair, modelar e retornar um DataFrame de uma determinada página web."""
    
    #Defino um user-agent e faço a verificação padrão "robots.txt"
    headers = {"user-agent": "Mozilla/5.0"}
    
    #Recebo a url que terá os seus dados coletados e conecto a mesma 
    url = requests.get(url_search, headers = headers)
    fonte = url.text
    
    #Chamo BeautifulSoup e organizo os dados coletados
    soup = BeautifulSoup(fonte, "lxml-xml")
    
    #Faço algumas extrações específicas utilizando uma Expressão Regular
    categorias = soup.find_all("a", {"href": re.compile("catalogue/category/books/.*/index.html")})
    preco = soup.find_all("p", {"class":"price_color"})
    estoque = soup.find_all("p", {"class": "instock availability"})
    
    avaliacao=[]
    
    #Crio um iterador a parte para extrair uma tag que requer um pouco mais de 'atenção'
    for a in soup.find_all("p", class_=True):
        avaliacao.append(a["class"])    
    for a in avaliacao:
        if a == 'price_color' and 'instock availability':
            avaliacao.remove('price_color')
            avaliacao.remove('instock availability')
    
    lista_analises=[]
    
    #Realizo uma iteração para limpar e modelar os dados
    for c, p, e, t in zip(categorias, preco, estoque, soup.find_all("a", title=True)):
        lista_categorias = c.text
        lista_categorias = lista_categorias.replace("\n","")

        lista_precos = p.contents[0].text
        lista_precos = lista_precos.replace("Â£", "").replace(".","")

        status_estoque = e.text
        status_estoque = status_estoque.replace("\n","")

        lista_analises.append((lista_categorias.strip(), lista_precos.strip(), 
                               status_estoque.strip(), t["title"].strip()))
    
    #Crio um DataFrame e incluo os dados extraídos e modelados
    df_analise_web = pd.DataFrame(lista_analises, columns=["Categoria","Preco","Estoque","Titulo"])
    df_analise_web["Preco"] = df_analise_web["Preco"].astype(np.float32)
    df_analise_web["Avaliacao"] = avaliacao

    return df_analise_web

In [3]:
df_analise = web_search("http://books.toscrape.com/")
df_analise

,Categoria,Preco,Estoque,Titulo,Avaliacao
0,Travel,5177.0,In stock,A Light in the Attic,star-rating Three
1,Mystery,5374.0,In stock,Tipping the Velvet,star-rating One
2,Historical Fiction,5010.0,In stock,Soumission,star-rating One
3,Sequential Art,4782.0,In stock,Sharp Objects,star-rating Four
4,Classics,5423.0,In stock,Sapiens: A Brief History of Humankind,star-rating Five
5,Philosophy,2265.0,In stock,The Requiem Red,star-rating One
6,Romance,3334.0,In stock,The Dirty Little Secrets of Getting Your Dream...,star-rating Four
7,Womens Fiction,1793.0,In stock,The Coming Woman: A Novel Based on the Life of...,star-rating Three
8,Fiction,2260.0,In stock,The Boys in the Boat: Nine Americans and Their...,star-rating Four
9,Childrens,5215.0,In stock,The Black Maria,star-rating One


#### Feito por mim, <a href="https://linkedin.com/in/mario-barcelos"> Mario Barcelos! </a> 